In [1]:
import tkinter as tk
import time
import winsound
import random
import csv
import threading
import scipy
import numpy as np
import pandas as pd
from datetime import datetime
from pylsl import StreamInlet, resolve_stream

#how many directions to appear on the screen
num_trial=15

count = 0
# Create an empty list to store the trial labels
trial_labels = ['']*num_trial

# Create the main window
root = tk.Tk()
root.geometry("500x500")
root.title("Task")
root.attributes('-fullscreen', True)  # Make the GUI full-screen

# Define the functions for displaying the cross and arrow
def show_cross():
    cross = tk.Label(root, text="+", font=("Arial", 250), width=10)
    cross.place(relx=0.5, rely=0.5, anchor="center")
    root.update()
    time.sleep(2)
    # Play the beep sound
    winsound.Beep(1000, 500)
    time.sleep(1)
    cross.destroy()

def show_arrow(direction):
    global count
    
    trial_labels[count]='Right' if direction == '→' else 'Left'
    count += 1
    arrow = tk.Label(root, text=direction, font=("Arial", 250), width=10)
    arrow.place(relx=0.5, rely=0.45, anchor="center")
    root.update()
    time.sleep(4)
    arrow.destroy()
    clear_screen()
    root.update()


def lsl_unicorn():
        # initialize the streaming layer
        finished = False
        streams = resolve_stream()
        inlet = StreamInlet(streams[0])
        # initialize the colomns of your data and your dictionary to capture the data.
        columns=['Time','FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8','AccX','AccY','AccZ',
        'Gyro1','Gyro2','Gyro3', 'Battery','Counter','Validation']
        data_dict = dict((k, []) for k in columns)
        time_data=[]
        while not finished:
           start_time = datetime.timestamp(datetime.now())
           time_data.append([start_time])
           # get the streamed data. Columns of sample are equal to the columns variable, only the first element being timestamp
           # concatenate timestamp and data in 1 list
           data, timestamp = inlet.pull_sample()
           all_data = [timestamp] + data
           # updating data dictionary with newly transmitted samples   
           i = 0
           for key in list(data_dict.keys()):
              data_dict[key].append(all_data[i])
              i = i + 1
           # data is collected at 250 Hz. Let's stop data collection after 9*num_trial seconds. Meaning we stop when we collected 250*9*num_trial samples.
           if len(data_dict['Time']) >= 250*9*(num_trial):
              finished = True
              with open('eeg_time.csv', mode='w', newline='') as file:
                  writer = csv.writer(file)
                  writer.writerow(["Start Time"])  # Write header row
                  writer.writerows(time_data)  # Write data rows
                # lastly, we can save our data to a CSV format.
        data_df = pd.DataFrame.from_dict(data_dict)
        data_df.to_csv('EEGdata.csv', index = False) 


time_data = []
# Define the function for the task
def task():
    total_start_time = time.time()
    
    for i in range(num_trial):
        start_time = datetime.timestamp(datetime.now())
        # Show the cross
        show_cross()
        global time_data
        # Show the arrow
        direction = random.choice(["←", "→"])
        show_arrow(direction)
        
        # Rest period
        time.sleep(1.5)
        elapsed_time = time.time() - start_time
        print("Elapsed time: {:.2f} seconds".format(elapsed_time))
        end_time = datetime.timestamp(datetime.now())
        time_data.append([start_time, end_time])

    

    total_elapsed_time = time.time() - total_start_time
    print("Elapsed time: {:.2f} seconds".format(total_elapsed_time))

    # Show the "Start Task" button again
    start_button = tk.Button(root, text="Start Task", font=("Arial", 30), bg="#47c77a", fg="black", bd=1, highlightthickness=0, padx=20, pady=10, command=start_task, relief=tk.SOLID)
    start_button.place(relx=0.5, rely=0.5, anchor="center")
    
    # Create the exit button
    exit_button = tk.Button(root, text="X", font=("Arial", 8), bg="#ff6b6b", fg="black", bd=0, highlightthickness=0, padx=4, pady=4, command=root.destroy, relief=tk.SOLID)
    exit_button.place(relx=0.99999, rely=0.00001, anchor="ne")

# Define the function to start the task
def start_task():
    start_button.destroy()  # Remove the "Start Task" button
    exit_button.destroy()
    
    # Create a new thread to run the task function
    task_thread = threading.Thread(target=lsl_unicorn)
    task_thread.start()
    time.sleep(1)
    task()  # Run the task

    # Save the trial labels to a CSV file
    with open("trial_labels.csv", "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["trial", "direction"])
        for i, label in enumerate(trial_labels):
            writer.writerow([i+1, label])

    csv_filename = "time_data_GUI.csv"
    global time_data
    # Save the time_data list to the CSV file
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Start Time", "End Time"])  # Write header row
        writer.writerows(time_data)  # Write data rows

# Define the function to clear the screen
def clear_screen():
    for widget in root.winfo_children():
        widget.destroy()


# Create the "Start Task" button
start_button = tk.Button(root, text="Start Task", font=("Arial", 30), bg="#47c77a", fg="black", bd=1, highlightthickness=0, padx=20, pady=10, command=start_task, relief=tk.SOLID)
start_button.place(relx=0.5, rely=0.5, anchor="center")

# Create the exit button
exit_button = tk.Button(root, text="X", font=("Arial", 8), bg="#ff6b6b", fg="black", bd=0, highlightthickness=0, padx=4, pady=4, command=root.destroy, relief=tk.SOLID)
exit_button.place(relx=0.99999, rely=0.00001, anchor="ne")


# Center the arrow labels
def center_arrows():
    root.update_idletasks()  # Wait until the window is fully drawn
    for widget in root.winfo_children():
        if widget.cget("text") in ["←", "→"]:
            widget.place_configure(relx=0.5, rely=0.45, anchor="center")
            widget.config(width=10)

root.after(0, center_arrows)  # Call the center_arrows function after the window is drawn

# Start the GUI event loop
root.mainloop()


ModuleNotFoundError: No module named 'pylsl'